In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import keras

from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

### Get the data and pre-process it

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
# def plot_input_image(i):
#   plt.imshow(X_train[i], cmap='binary')
#   plt.title(y_train[i])
#   plt.show()

# for i in range(10):
  # plot_input_image(i)

#### Pre-process the images

In [ ]:
X_train.shape

(60000, 28, 28)

In [ ]:
# Normalizing the images to [0,1] range
X_train = X_train.astype(np.float32)/255  
X_test = X_test.astype(np.float32)/255

# Reshape/expand the dimensions of images to (28,28,1)
X_train = np.expand_dims(X_train, -1) # -1 is the index number, i.e. at the last
X_test = np.expand_dims(X_test, -1)

# Converting classes to One_Hot vector
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
X_train.shape

(60000, 28, 28, 1)

#### Building the model

In [ ]:
model = Sequential()

# adding 32 filtered convolutional layer
model.add(Conv2D(32, (3,3), input_shape = (28,28,1), activation='relu'))
model.add(MaxPool2D((2,2)))

# adding 64 filtered convolutional layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

# to prevent overfitting
model.add(Dropout(0.25))  

# to do classification, 10 is given because there are 10 categories 0-9
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [ ]:
model.compile(optimizer="adam", loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint

# EarlyStopping
es = EarlyStopping(monitor="val_acc", min_delta=0.01, patience=4, verbose=1)

# ModelCheckpoint
mc = ModelCheckpoint(F"./best_model.h5", monitor='val_acc', verbose=1, save_best_only=True)

cb = [es,mc]

#### Model Training

In [ ]:
his = model.fit(X_train, y_train, epochs=5, validation_split=0.3)

Epoch 1/5
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0334 - accuracy: 0.9893 - val_loss: 0.0445 - val_accuracy: 0.9864
Epoch 2/5
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0277 - accuracy: 0.9913 - val_loss: 0.0436 - val_accuracy: 0.9868
Epoch 3/5
1313/1313 [==============================] - 6s 5ms/step - loss: 0.0248 - accuracy: 0.9921 - val_loss: 0.0410 - val_accuracy: 0.9879
Epoch 4/5
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0235 - accuracy: 0.9925 - val_loss: 0.0508 - val_accuracy: 0.9866
Epoch 5/5
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0201 - accuracy: 0.9935 - val_loss: 0.0384 - val_accuracy: 0.9901


In [ ]:
from keras.models import save_model
model.save('/content/gdrive/My Drive/PROJECTS/HANDWRITTEN_DIGIT_RECOGNITION/best_model.h5')

In [ ]:
model_S = keras.models.load_model(F"/content/gdrive/My Drive/PROJECTS/HANDWRITTEN_DIGIT_RECOGNITION/best_model.h5")

In [ ]:
score = model_S.evaluate(X_test, y_test)

# score[0] returns the loss and score[1] returns the accuracy
print(f"The model accuracy is {score[1]}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0291 - accuracy: 0.9914
The model accuracy is 0.9914000034332275
